In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..', 'external', 'MiDaS')))

In [3]:
%env CUDA_VISIBLE_DEVICES=0,1

env: CUDA_VISIBLE_DEVICES=0,1


In [4]:
%env MIX6_DATASETS=/runai-ivrl-scratch/students/2021-fall-sp-jellouli/mix6

env: MIX6_DATASETS=/runai-ivrl-scratch/students/2021-fall-sp-jellouli/mix6


In [5]:
from midas_hkrm.optim import MidasHKRMTrainer, SSITrimmedMAELoss

In [6]:
from midas_hkrm.utils import midas_train_transform, midas_eval_transform

from midas_hkrm.utils import setup_logger

from torch.optim import Adam
from midas_hkrm.depth import create_midas_hkrm_model
from midas_hkrm.datasets import HRWSI, ApolloScape, BlendedMVS, MegaDepth, RedWeb, TartanAir

In [7]:
setup_logger()

In [8]:
model = create_midas_hkrm_model(10, 0.4, object_model_weights="/runai-ivrl-scratch/students/2021-fall-sp-jellouli/output/model_final.pth")

INFO:fvcore.common.checkpoint:[Checkpointer] Loading from /runai-ivrl-scratch/students/2021-fall-sp-jellouli/output/model_final.pth ...


Using cache found in /runai-ivrl-scratch/students/2021-fall-sp-jellouli/.cache/torch/hub/facebookresearch_WSL-Images_main


INFO:midas_hkrm.depth.midas_hkrm:Loading pretrained model_weights
INFO:midas_hkrm.depth.midas_hkrm:Randomnly intializing missing model weights


In [9]:
for m in model.backbone.parameters():
        m.requires_grad = False

In [10]:
datasets = [
            ApolloScape(),
            TartanAir(),
            RedWeb(),
            BlendedMVS(),
            MegaDepth(),
            HRWSI(),
        ]


In [11]:
criterion = SSITrimmedMAELoss()
optimizer = Adam(
            [{"params": model.channel_reduc.parameters(), "lr": 1e-4}],
            lr=1e-2,
            betas=(0.9, 0.999),
        )

In [12]:
trainer = MidasHKRMTrainer(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    datasets=datasets,
    train_transform=midas_train_transform,
    test_transform=midas_eval_transform,
    save=True,
    save_path='.',
    test=True,
    test_split_size=0.01,
    same_test_size=True,
    test_after=1,
    save_after=1,
    batch_size=6,
    test_batch_size=6,
    device="cuda",
    max_iter=1,
)

INFO:midas_hkrm.optim.trainer:Trainer initialized on datasets: ['ApolloScape', 'TartanAir', 'ReDWeb_V1', 'BlendedMVS', 'MegaDepth_v1', 'HR-WSI']
INFO:midas_hkrm.optim.trainer:Preparing test sets
INFO:midas_hkrm.optim.trainer:Created a test set for dataset ApolloScape of size 36
INFO:midas_hkrm.optim.trainer:Created a test set for dataset TartanAir of size 36
INFO:midas_hkrm.optim.trainer:Created a test set for dataset ReDWeb_V1 of size 36
INFO:midas_hkrm.optim.trainer:Created a test set for dataset BlendedMVS of size 36
INFO:midas_hkrm.optim.trainer:Created a test set for dataset MegaDepth_v1 of size 36
INFO:midas_hkrm.optim.trainer:Created a test set for dataset HR-WSI of size 36


In [13]:
trainer.train()

INFO:midas_hkrm.optim.trainer:Saving initial trainer state
INFO:midas_hkrm.optim.trainer:Saved state state_0.tar
INFO:midas_hkrm.optim.trainer:Starting at iteration 0


  0%|                                                                  | 0/1 [00:00<?, ?it/s]

INFO:midas_hkrm.optim.trainer:Training iteration: 1
INFO:midas_hkrm.optim.trainer:Starting evaluation at step 1


  0%|                                                                  | 0/1 [00:46<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.39 GiB (GPU 0; 11.91 GiB total capacity; 8.52 GiB already allocated; 1.16 GiB free; 10.00 GiB reserved in total by PyTorch)